In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2, 3"

In [8]:
!conda install -c conda-forge numpy=1.23.1
!pip install -q  torch peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 accelerate

Retrieving notices: ...working... done
done
Solving environment: unsuccessful initial attempt using frozen solve. Retrying with flexible solve.
^C
ERROR: Operation cancelled by user


In [1]:
!pip install transformers peft bitsandbytes

In [2]:
pip install protobuf==3.20.*

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig
from trl import SFTTrainer

2023-11-09 18:48:39.815198: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /path/to/anaconda/envs/myenv/lib:/usr/local/cuda/lib64:/usr/local/cuda-11.7/lib64::/usr/local/cuda/lib64:/usr/local/cuda-11.7/lib64
/home/sjbuhan/anaconda3/lib/python3.8/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2,3"
# Dataset
data_name = "aeoebe/josun"
training_data = load_dataset(data_name, split="train", on_bad_lines='skip')

# Model and tokenizer names
base_model_name = "meta-llama/Llama-2-13b-chat-hf"
refined_model = "ko-llama-2-13b-josun-finetune" #You can give it your own name

# Tokenizer
llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"  # Fix for fp16

# Quantization Config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

# Model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map={"": 0}
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
torch.cuda.empty_cache()

In [5]:
# LoRA Config
peft_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM"
)

# Training Params
train_params = TrainingArguments(
    output_dir="./results_modified",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

# Trainer
fine_tuning = SFTTrainer(
    model=base_model,
    train_dataset=training_data,
    peft_config=peft_parameters,
    dataset_text_field="공조 정랑 조목이 퇴직을 청하다",
    tokenizer=llama_tokenizer,
    args=train_params
)

# Training
fine_tuning.train()

# Save Model
fine_tuning.model.save_pretrained(refined_model)

/home/sjbuhan/anaconda3/lib/python3.8/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/sjbuhan/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
25,2.247000
50,1.942000
75,1.764100
100,1.731700
125,1.717000
150,1.634500
175,1.622300
200,1.534000
225,1.615100
250,1.508100


/home/sjbuhan/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sjbuhan/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sjbuhan/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sjbuhan/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, bu

In [7]:
# Generate Text
query = "옥당이 시폐 상소"
text_gen = pipeline(task="text-generation", model=refined_model, tokenizer=llama_tokenizer, max_length=200)
output = text_gen(f"<s>[INST] {query} [/INST]")
print(output[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

<s>[INST] 옥당이 시폐 상소 [/INST]  Here are some possible ways to address the issue of a colleague who is not pulling their weight in the workplace:

1. Communicate directly and openly: It's important to address the issue directly with the colleague and not through gossip or rumors. Be specific about the behaviors or tasks that are causing concern and provide examples.
2. Use "I" statements: Instead of accusing the colleague of not pulling their weight, use "I" statements to express your concerns. For example, "I've noticed that you haven't been completing your tasks on time" rather than "You're not doing your job."
3. Focus on the specific behaviors, not the person: Avoid making personal attacks or generalizations about the colleague's abilities. Instead, focus on the


In [26]:
prompt = "Below is an incomplete sentence. Appropriately predict the next token of the sentence and complete the sentence."
### Text: %s ### Response:

def gen(x):
    q = prompt % (x,)
    gened = base_model.generate(
        **llama_tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=2048,
        early_stopping=True,
        do_sample=True,
    )
    return llama_tokenizer.decode(gened[0]).replace(q, "")

In [29]:
gen("옥당이 시폐 상소")

'<s> 옥당이 시폐 상소를 올리다. 그들의 의견을 따르다. ### Text: 옥당이 죄를 청하다. ### Response: 옥당이 죄를 청하다. 그들의 의견을 따르다.</s>'

In [25]:
gen("헌부가 시약청에 내린 상이 과중")

'<s> 헌부의 죄를 다스리도록 하다. 헌부의 출사를 명하다. 헌부의 계사를 올리지 말도록 하다.</s>'

In [20]:
gen("헌부가 흉년이 들었으니 천금을 위 망을 치는 일을")

'<s> 흉년이 심하다고 잡아 올리다 않으면 형을 가하라고 하다. 이는 예조에서 아뢰다 처리하게 하다.</s>'

In [33]:
gen("이조 판서 이산해가 신병으로 정사하자")

'<s> 물러가도록 돈독하다. 그러니 물러가도록 당부하다. 약을 보내다. 접견하다. 치료하다. 술을 내리다. 반열에 나가다. 풍양에 가다. 몸소 치료하다. 약을 질려 주다. 매일 약을 입히다. 약으로 치료하다. 접견하다. 약을 올리다. 약을 쓰다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리다. 약을 올리